# ASCII

ASCII is a 7-bit encoding standard which allows the representation of text using the integers 0-127.

Using the below integer array, convert the numbers to their corresponding ASCII characters to obtain a flag.

`[99, 114, 121, 112, 116, 111, 123, 65, 83, 67, 73, 73, 95, 112, 114, 49, 110, 116, 52, 98, 108, 51, 125]`

> In Python, the `chr()` function can be used to convert an ASCII ordinal number to a character (the `ord()` function does the opposite).

## Solution

In [1]:
flag = bytes([99, 114, 121, 112, 116, 111, 123, 65, 83, 67, 73, 73, 95, 112, 114, 49, 110, 116, 52, 98, 108, 51, 125]).decode()
print("Flag: " + flag)

Flag: crypto{ASCII_pr1nt4bl3}


### Flag: crypto{ASCII_pr1nt4bl3}

# Hex

When we encrypt something the resulting ciphertext commonly has bytes which are not printable ASCII characters. If we want to share our encrypted data, it's common to encode it into something more user-friendly and portable across different systems.

Hexadecimal can be used in such a way to represent ASCII strings. First each letter is converted to an ordinal number according to the ASCII table (as in the previous challenge). Then the decimal numbers are converted to base-16 numbers, otherwise known as hexadecimal. The numbers can be combined together, into one long hex string.

Included below is a flag encoded as a hex string. Decode this back into bytes to get the flag.

`63727970746f7b596f755f77696c6c5f62655f776f726b696e675f776974685f6865785f737472696e67735f615f6c6f747d`

> In Python, the `bytes.fromhex()` function can be used to convert hex to bytes. The `.hex()` instance method can be called on byte strings to get the hex representation.


Resources:
  - [ASCII table](https://www.rapidtables.com/code/text/ascii-table.html)
  - [Wikipedia: Hexadecimal](https://en.wikipedia.org/wiki/Hexadecimal)

## Solution

In [2]:
flag = bytes.fromhex("63727970746f7b596f755f77696c6c5f62655f776f726b696e675f776974685f6865785f737472696e67735f615f6c6f747d").decode()
print("Flag: " + flag)

Flag: crypto{You_will_be_working_with_hex_strings_a_lot}


### Flag: crypto{You_will_be_working_with_hex_strings_a_lot}

# Base64

Another common encoding scheme is Base64, which allows us to represent binary data as an ASCII string using an alphabet of 64 characters. One character of a Base64 string encodes 6 binary digits (bits), and so 4 characters of Base64 encode three 8-bit bytes.

Base64 is most commonly used online, so binary data such as images can be easily included into HTML or CSS files.

Take the below hex string, decode it into bytes and then encode it into Base64.

`72bca9b68fc16ac7beeb8f849dca1d8a783e8acf9679bf9269f7bf`

> In Python, after importing the base64 module with `import base64`, you can use the `base64.b64encode()` function. Remember to decode the hex first as the challenge description states.

## Solution

In [5]:
from base64 import b64encode

flag = b64encode(bytes.fromhex("72bca9b68fc16ac7beeb8f849dca1d8a783e8acf9679bf9269f7bf"))
print("Flag: " + flag.decode())

Flag: crypto/Base+64+Encoding+is+Web+Safe/


### Flag: crypto/Base+64+Encoding+is+Web+Safe/

# Bytes and Big Integers

Cryptosystems like RSA works on numbers, but messages are made up of characters. How should we convert our messages into numbers so that mathematical operations can be applied?

The most common way is to take the ordinal bytes of the message, convert them into hexadecimal, and concatenate. This can be interpreted as a base-16/hexadecimal number, and also represented in base-10/decimal.

To illustrate:

> message: HELLO
> ascii bytes: [72, 69, 76, 76, 79]
> hex bytes: [0x48, 0x45, 0x4c, 0x4c, 0x4f]
> base-16: 0x48454c4c4f
> base-10: 310400273487

> Python's PyCryptodome library implements this with the methods `bytes_to_long()` and `long_to_bytes()`. You will first have to install PyCryptodome and import it with `from Crypto.Util.number import *`. For more details check the [FAQ](https://cryptohack.org/faq/#install).


Convert the following integer back into a message:

`11515195063862318899931685488813747395775516287289682636499965282714637259206269`

## Solution

In [6]:
from Crypto.Util.number import long_to_bytes

flag = long_to_bytes(11515195063862318899931685488813747395775516287289682636499965282714637259206269).decode()
print("Flag: " + flag)

Flag: crypto{3nc0d1n6_4ll_7h3_w4y_d0wn}


### Flag: crypto{3nc0d1n6_4ll_7h3_w4y_d0wn}

# Encoding Challenge

Now you've got the hang of the various encodings you'll be encountering, let's have a look at automating it.

Can you pass all 100 levels to get the flag?

The *13377.py* file attached below is the source code for what's running on the server. The pwntools_example.py file provides the start of a solution using the incredibly convenient pwntools library. which we recommend. If you'd prefer to use Python's in-built telnetlib, telnetlib_example.py is also provided.

For more information about connecting to interactive challenges, see the [FAQ](https://cryptohack.org/faq#netcat). Feel free to skip ahead to the cryptography if you aren't in the mood for a coding challenge!

Connect at nc `socket.cryptohack.org 13377`

Challenge files:
  - [13377.py](https://cryptohack.org/static/challenges/13377_43de0a0efed6ed7bd890d1c79db22fb1.py)
  - [pwntools_example.py](https://cryptohack.org/static/challenges/pwntools_example_f93ca6ccef2def755aa8f6d9aa6e9c5b.py)
  - [telnetlib_example.py](https://cryptohack.org/static/challenges/telnetlib_example_5b11a835055df17c7c8f8f2a08782c44.py)

## Challenge overview

### 13377.py

```python
#!/usr/bin/env python3

from Crypto.Util.number import bytes_to_long, long_to_bytes
from utils import listener # this is cryptohack's server-side module and not part of python
import base64
import codecs
import random

FLAG = "crypto{????????????????????}"
ENCODINGS = [
    "base64",
    "hex",
    "rot13",
    "bigint",
    "utf-8",
]
with open('/usr/share/dict/words') as f:
    WORDS = [line.strip().replace("'", "") for line in f.readlines()]


class Challenge():
    def __init__(self):
        self.challenge_words = ""
        self.stage = 0

    def create_level(self):
        self.stage += 1
        self.challenge_words = "_".join(random.choices(WORDS, k=3))
        encoding = random.choice(ENCODINGS)

        if encoding == "base64":
            encoded = base64.b64encode(self.challenge_words.encode()).decode() # wow so encode
        elif encoding == "hex":
            encoded = self.challenge_words.encode().hex()
        elif encoding == "rot13":
            encoded = codecs.encode(self.challenge_words, 'rot_13')
        elif encoding == "bigint":
            encoded = hex(bytes_to_long(self.challenge_words.encode()))
        elif encoding == "utf-8":
            encoded = [ord(b) for b in self.challenge_words]

        return {"type": encoding, "encoded": encoded}

    #
    # This challenge function is called on your input, which must be JSON
    # encoded
    #
    def challenge(self, your_input):
        if self.stage == 0:
            return self.create_level()
        elif self.stage == 100:
            self.exit = True
            return {"flag": FLAG}

        if self.challenge_words == your_input["decoded"]:
            return self.create_level()

        return {"error": "Decoding fail"}


listener.start_server(port=13377)
```

## Solution

In [ ]:
from tqdm import tqdm
from pwn import *
from Crypto.Util.number import long_to_bytes
from base64 import b64decode
import json
from string import ascii_lowercase
from codecs import decode

class challenge:
    def __init__(self, HOST, PORT) -> None:
        self.io = remote(HOST, PORT)
    
    def solve(self):
        chall = json.loads(self.io.recvline().strip().decode())
        
        if chall['type'] == 'base64':
            answer = {'decoded': b64decode(chall['encoded']).decode()}
            self.io.sendline(json.dumps(answer).encode())
        
        elif chall['type'] == 'bigint':
            answer = {'decoded': long_to_bytes(int(chall['encoded'], 16)).decode()}
            self.io.sendline(json.dumps(answer).encode())
        
        elif chall['type'] == 'hex':
            answer = {'decoded': bytes.fromhex(chall['encoded']).decode()}
            self.io.sendline(json.dumps(answer).encode())
        
        elif chall['type'] == 'utf-8':
            answer = {'decoded': bytes(chall['encoded']).decode()}
            self.io.sendline(json.dumps(answer).encode())
        
        else:
            answer = {'decoded': decode(chall['encoded'], 'rot_13')}
            self.io.sendline(json.dumps(answer).encode())

    def interactive(self):
        return self.io.interactive()
    
if __name__ == '__main__':
    HOST = "socket.cryptohack.org"
    PORT = 13377
    chall = challenge(HOST, PORT)
    
    for i in tqdm(range(100)):
        chall.solve()
    
    chall.interactive()

In [ ]:
  [14:38:23] [~/Desktop/Github/CTF-Wargames/CryptoHack/GENERAL/ENCODING] ❱❱❱ python encoding_challenge.py
[+] Opening connection to socket.cryptohack.org on port 13377: Done
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:16<00:00,  1.96s/it]
[*] Switching to interactive mode
{"flag": "crypto{3nc0d3_d3c0d3_3nc0d3}"}
[*] Got EOF while reading in interactive
$ 
[*] Interrupted
[*] Closed connection to socket.cryptohack.org port 13377

### Flag: crypto{3nc0d3_d3c0d3_3nc0d3}